<a href="https://colab.research.google.com/github/anehrani/Tutorials/blob/main/mathopt_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mathopt
!pip install ortools

In [3]:
from absl import app
import numpy as np

from ortools.math_opt.python import mathopt

In [7]:
"""
 Function for getting the initial information of the Boxes
"""
def get_bin_info():
    """
        Columns: id, quantity, length, width, height, max weight, COM_x, COM_y, COM_z


    """
    # assuming data is stored as follows in csv file
    bins_data = [
        [0, 2, 900, 900, 900, 800, 750, 750, 0],
        #[1, 2, 700, 700, 700, 500, 250, 250, 0],

    ]
    bins = []

    for b in bins_data:
        for _ in range(b[1]):
            bins.append( b )

    return np.array(bins)



"""
Function for getting the initial information of the items
"""
def get_item_info():
    """
        Columns are: id, quantity, length, width, height, weight, family, incompatibilities, stackability_below, stackability_above, rotatability_X, rotatability_Y, rotatability_Z

        stackability_below: if 1 cant put any item on top of this item
        stackability_above: if 1 cant put this item on top of any other item

        data is similar to the csv file structure
    """
    data_csv = [
                [0, 6, 118, 147, 216, 50, 1, 0, 0, 0, 0, 0, 0],
                [1, 8, 115, 165, 264, 20, 0, 0, 0, 0, 0, 0, 0],
                [2, 8, 120, 196, 267, 20, 0, 0, 0, 0, 0, 0, 0],
                [3,12, 171, 107, 301, 20, 1, 0, 0, 0, 0, 0, 0],
                [4, 7, 180, 118, 298, 20, 0, 0, 0, 0, 0, 0, 0],
                [5, 9, 165, 121, 338, 20, 0, 0, 0, 0, 0, 0, 0],
                [6,15, 185, 149, 257, 20, 0, 0, 0, 0, 0, 0, 0],
                [7,15, 197, 158, 251, 50, 0, 0, 0, 0, 0, 0, 0],
                [8, 6, 107, 162, 207, 20, 1, 0, 0, 0, 0, 0, 0],
                [9, 8, 101, 199, 296, 20, 0, 0, 0, 0, 0, 0, 0],
            ]
    # data_csv = [
    #             [0, 6, 218, 247, 216, 50, 1, 0, 0, 0, 0, 0, 0],
    #             [1, 2, 215, 265,  64, 20, 0, 0, 0, 0, 0, 0, 0],
    #             [2, 3, 220, 296, 267, 20, 0, 0, 0, 0, 0, 0, 0],
    #             [3, 6, 171, 307, 101, 20, 1, 0, 0, 0, 0, 0, 0],
    #             [4, 1, 280, 318, 298, 20, 0, 0, 0, 0, 0, 0, 0],
    #             [5, 2, 265, 321, 138, 20, 0, 0, 0, 0, 0, 0, 0],
    #             [6, 3, 185, 349, 157, 20, 0, 0, 0, 0, 0, 0, 0],
    #             [7, 5, 297, 358, 151, 50, 0, 0, 0, 0, 0, 0, 0],
    #             [8, 6, 207, 362, 107, 20, 1, 0, 0, 0, 0, 0, 0],
    #             [9, 4, 201, 399,  96, 20, 0, 0, 0, 0, 0, 0, 0],
    #         ]

    # setting the incompatibilities
    incompatibilities = np.zeros((len(data_csv), len(data_csv)) )
    for k in range(incompatibilities.shape[0]):
        for i in range(k):
            if data_csv[i][7] == data_csv[k][7] and data_csv[i][7] > 0:
                incompatibilities[i, k] = 1

    # setting the alpha values for the family constraint
    alpha = {}


    # parsing data <this part to be implemented later >
    data_mat = []
    for b in data_csv:
        for i in range(b[1]):
            data_mat.append( [i] + b )
    data_mat = np.array(data_mat)

    families = set(map(int, data_mat[:, 7]))
    N = data_mat.shape[0]
    for f in families:
        for i in range(N):
            alpha[(i, f)] = 1 if data_mat[i][7] == f else 0


    return data_mat, alpha, incompatibilities, families, data_mat[:,9], data_mat[:,10], data_mat[:,11], data_mat[:,12], data_mat[:,13]


In [11]:
def define_variables(hsmodel , constants: dict ):


    # adding p_{ij} and u_{j}
    for j  in range(constants["num_bins"]):
        hsmodel.add_binary_variable (name= "u_" + str(j) )
        for i in range(constants["num_boxes"]):
            hsmodel.add_binary_variable(name= "p_" + str(i) + "_" + str(j) )

    # adding x^{+}_{ik} and x^{-}_{ik}, y^{+}_{ik} and y^{-}_{ik}, z^{+}_{ik} and z^{-}_{ik}
    # these are the binary variables that indicate the order of the item i in the bin k
    # NOTE: always second one is larger index (due to symmetry only upper triangular is considered)
    for i in range(constants["num_boxes"]):
        for k in range(i, constants["num_boxes"]):
            hsmodel.add_binary_variable(name= "x_p_" + str(i) + "_" + str(k) )
            hsmodel.add_binary_variable(name= "x_n_" + str(i) + "_" + str(k) )
            hsmodel.add_binary_variable(name= "y_p_" + str(i) + "_" + str(k) )
            hsmodel.add_binary_variable(name= "y_n_" + str(i) + "_" + str(k) )
            hsmodel.add_binary_variable(name= "z_p_" + str(i) + "_" + str(k) )
            hsmodel.add_binary_variable(name= "z_n_" + str(i) + "_" + str(k) )

    # adding the orientation variables
    for i in range(constants["num_boxes"]):
        hsmodel.add_binary_variable(name= "lx_" + str(i)  )
        hsmodel.add_binary_variable(name= "ly_" + str(i)  )
        hsmodel.add_binary_variable(name= "lz_" + str(i)  )
        #
        hsmodel.add_binary_variable(name= "wx_" + str(i)  )
        hsmodel.add_binary_variable(name= "wy_" + str(i)  )
        hsmodel.add_binary_variable(name= "wz_" + str(i)  )
        #
        hsmodel.add_binary_variable(name= "hx_" + str(i)  )
        hsmodel.add_binary_variable(name= "hy_" + str(i)  )
        hsmodel.add_binary_variable(name= "hz_" + str(i)  )


    # adding family variables (in this specific example there is only one family)
    for j in range(constants["num_bins"]):
        for f in range( constants["num_families"] ):
            hsmodel.add_binary_variable(name= "beta_" + str(j) + "_" + str(f) )

    # adding the continuous variables (xyz coordinates of the items)
    for i in range(constants["num_boxes"]):
        hsmodel.add_variable(name= "x_" + str(i)) # L
        hsmodel.add_variable(name= "y_" + str(i)) # W
        hsmodel.add_variable(name= "z_" + str(i)) # H


    # adding the continuous variables (gx gy gz coordinates of the bins)
    for j in range(constants["num_bins"]):
        hsmodel.add_variable(name= "gx_" + str(j))
        hsmodel.add_variable(name= "gy_" + str(j))
        hsmodel.add_variable(name= "gz_" + str(j))
        # extra variables
        hsmodel.add_variable(name= "ex_p_" + str(j), lb=0, ub=constants["M"])
        hsmodel.add_variable(name= "ex_n_" + str(j), lb=0, ub=constants["M"])
        hsmodel.add_variable(name= "ey_p_" + str(j), lb=0, ub=constants["M"])
        hsmodel.add_variable(name= "ey_n_" + str(j), lb=0, ub=constants["M"])
        hsmodel.add_variable(name= "ez_p_" + str(j), lb=0, ub=constants["M"])
        hsmodel.add_variable(name= "ez_n_" + str(j), lb=0, ub=constants["M"])

    # variables for center of gravity handling
    for j in range( constants["num_bins"] ):
        for i in range(constants["num_boxes"]):
            hsmodel.add_variable(name= "c_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "d_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "e_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "t_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "v_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "m_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "n_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_binary_variable(name= "b_" + str(i) + "_" + str(j) )
            hsmodel.add_binary_variable(name= "r_" + str(i) + "_" + str(j) )
            hsmodel.add_binary_variable(name= "s_" + str(i) + "_" + str(j) )
            # newly added variables
            hsmodel.add_variable(name= "phx_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "phy_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "plz_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "pwz_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])
            hsmodel.add_variable(name= "phz_" + str(i) + "_" + str(j), lb=0, ub=constants["M"])

    return hsmodel

In [27]:
# large scalar
M = 100000

# number of bins
in_data, family_alpha, incompatibles, families, stackability_below, stackability_above, rot_x, rot_y, rot_z = get_item_info()
bins_info = get_bin_info()

J = bins_info.shape[0]
F = len(families)

# number of items
N : int = in_data.shape[0]

constants = {
        "num_bins": J,
        "num_boxes": N,
        "num_families": F,
        "M": M,
    }

In [9]:
model = mathopt.Model(name="Advanced linear programming example")

In [26]:
#v1 = model.variables()
#for item in v1:
    #print(item.name)
    #print(item.id)


In [14]:
model = define_variables( model, constants )

In [25]:
varsDict = {}
modelVars = model.variables()
for var in modelVars:
    varsDict[var.name] = var

In [30]:
## adding the constraints
#  ensure that an item i can be assigned to a bin j only if bin j is used EQ(4) pij ≤ uj ∀i ∈ I, ∀j ∈ J
for j in range(J):
    for i in range(N):
        model.add_linear_constraint( varsDict["p_" + str(i) + "_" + str(j) ] <= varsDict["u_" + str(j) ], name= "c1_" + str(i) + "_" + str(j),  )


In [31]:
    # ensure that each item i is packed only in one bin J
    for i in range(N):
        model.add_linear_constraint( np.sum( [ varsDict["p_" + str(i) + "_" + str(j) ] for j in range(J) ] ) <= 1, name= "c2_1_" + str(i), )


In [32]:
for j in range(J):
    for i in range(N):
        model.add_linear_constraint( varsDict["x_" + str(i) ] + in_data[i][3]*varsDict["lx_" + str(i) ] + in_data[i][4]*varsDict["wx_" + str(i) ] + in_data[i][5]*varsDict["hx_" + str(i) ] + 0.1 <= bins_info[j, 2] * varsDict["u_" + str(j) ] + (1 - varsDict["p_" + str(i) + "_" + str(j) ] ) * M, name= "c3x_" + str(i) + "_" + str(j), )
        model.add_linear_constraint( varsDict["y_" + str(i) ] + in_data[i][3]*varsDict["ly_" + str(i) ] + in_data[i][4]*varsDict["wy_" + str(i) ] + in_data[i][5]*varsDict["hy_" + str(i) ] + 0.1 <= bins_info[j, 3] * varsDict["u_" + str(j) ] + (1 - varsDict["p_" + str(i) + "_" + str(j) ] ) * M, name= "c3y_" + str(i) + "_" + str(j), )
        model.add_linear_constraint( varsDict["z_" + str(i) ] + in_data[i][3]*varsDict["lz_" + str(i) ] + in_data[i][4]*varsDict["wz_" + str(i) ] + in_data[i][5]*varsDict["hz_" + str(i) ] + 0.1 <= bins_info[j, 4] * varsDict["u_" + str(j) ] + (1 - varsDict["p_" + str(i) + "_" + str(j) ] ) * M, name= "c3z_" + str(i) + "_" + str(j), )


In [33]:
for k in range(N):
    for i in range(k):
        model.add_linear_constraint( varsDict["x_" + str(i) ] + in_data[i][3]* varsDict["lx_" + str(i) ] + in_data[i][4]* varsDict["wx_" + str(i) ] + in_data[i][5]* varsDict["hx_" + str(i) ] + 0.1 <= varsDict["x_" + str(k) ] + (1 - varsDict["x_p_" + str(i) + "_" + str(k) ] )*M , name= "c4xp_" + str(i) + "_" + str(k), )
        model.add_linear_constraint( varsDict["x_" + str(k) ] + in_data[k][3]* varsDict["lx_" + str(k) ] + in_data[k][4]* varsDict["wx_" + str(k) ] + in_data[k][5]* varsDict["hx_" + str(k) ] + 0.1 <= varsDict["x_" + str(i) ] + (1 - varsDict["x_n_" + str(i) + "_" + str(k) ] )*M , name= "c4xn_" + str(i) + "_" + str(k), )
        model.add_linear_constraint( varsDict["y_" + str(i) ] + in_data[i][3]* varsDict["ly_" + str(i) ] + in_data[i][4]* varsDict["wy_" + str(i) ] + in_data[i][5]* varsDict["hy_" + str(i) ] + 0.1 <= varsDict["y_" + str(k) ] + (1 - varsDict["y_p_" + str(i) + "_" + str(k) ] )*M , name= "c4yp_" + str(i) + "_" + str(k), )
        model.add_linear_constraint( varsDict["y_" + str(k) ] + in_data[k][3]* varsDict["ly_" + str(k) ] + in_data[k][4]* varsDict["wy_" + str(k) ] + in_data[k][5]* varsDict["hy_" + str(k) ] + 0.1 <= varsDict["y_" + str(i) ] + (1 - varsDict["y_n_" + str(i) + "_" + str(k) ] )*M , name= "c4yn_" + str(i) + "_" + str(k), )
        model.add_linear_constraint( varsDict["z_" + str(i) ] + in_data[i][3]* varsDict["lz_" + str(i) ] + in_data[i][4]* varsDict["wz_" + str(i) ] + in_data[i][5]* varsDict["hz_" + str(i) ] + 0.1 <= varsDict["z_" + str(k) ] + (1 - varsDict["z_p_" + str(i) + "_" + str(k) ] )*M , name= "c4zp_" + str(i) + "_" + str(k), )
        model.add_linear_constraint( varsDict["z_" + str(k) ] + in_data[k][3]* varsDict["lz_" + str(k) ] + in_data[k][4]* varsDict["wz_" + str(k) ] + in_data[k][5]* varsDict["hz_" + str(k) ] + 0.1 <= varsDict["z_" + str(i) ] + (1 - varsDict["z_n_" + str(i) + "_" + str(k) ] )*M , name= "c4zn_" + str(i) + "_" + str(k), )


In [34]:
for j in range(J):
    for k in range(N):
        for i in range(k):
            model.add_linear_constraint( varsDict["x_p_" + str(i) + "_" + str(k) ] + varsDict["x_n_" + str(i) + "_" + str(k) ] + varsDict["y_p_" + str(i) + "_" + str(k) ] + varsDict["y_n_" + str(i) + "_" + str(k) ] + varsDict["z_p_" + str(i) + "_" + str(k) ] + varsDict["z_n_" + str(i) + "_" + str(k) ] >= varsDict["p_" + str(i) + "_" + str(j) ] + varsDict["p_" + str(k) + "_" + str(j) ] - 1, name= "c5_" + str(i) + "_" + str(k), )


In [35]:
for j in range(J):
    for k in range(N):
        for i in range(k):
            if stackability_below[k] > 0:
                model.add_linear_constraint(
                    varsDict["z_p_" + str(i) + "_" + str(k) ] <= varsDict["x_p_" + str(i) + "_" + str(k) ] + varsDict["x_n_" + str(i) + "_" + str(k) ] + varsDict["y_p_" + str(i) + "_" + str(k) ] + varsDict["y_n_" + str(i) + "_" + str(k) ] + ( 2 - varsDict["p_" + str(k) + "_" + str(j) ] + varsDict["p_" + str(i) + "_" + str(j) ] ), name= "c5_1_" + str(i) + "_" + str(k), )
            if stackability_above[k] > 0:
                model.add_linear_constraint(
                    varsDict["z_n_" + str(i) + "_" + str(k) ] <= varsDict["x_p_" + str(i) + "_" + str(k) ] + varsDict["x_n_" + str(i) + "_" + str(k) ] + varsDict["y_p_" + str(i) + "_" + str(k) ] + varsDict["y_n_" + str(i) + "_" + str(k) ] + ( 2 - varsDict["p_" + str(k) + "_" + str(j) ] + varsDict["p_" + str(i) + "_" + str(j) ] ), name= "c5_2_" + str(i) + "_" + str(k), )



In [36]:
    # Constraints (16)-(19) define the proper orientations for each item ----->>> NEW CONSTRAINTS
    for i in range(N):
        model.add_linear_constraint( varsDict["lx_" + str(i) ] + varsDict["ly_" + str(i) ] + varsDict["lz_" + str(i) ] == 1, name= "c6_1_" + str(i), )
        model.add_linear_constraint( varsDict["wx_" + str(i) ] + varsDict["wy_" + str(i) ] + varsDict["wz_" + str(i) ] == 1, name= "c6_2_" + str(i), )
        model.add_linear_constraint( varsDict["hx_" + str(i) ] + varsDict["hy_" + str(i) ] + varsDict["hz_" + str(i) ] == 1, name= "c6_3_" + str(i), )
        model.add_linear_constraint( varsDict["lx_" + str(i) ] + varsDict["wx_" + str(i) ] + varsDict["hx_" + str(i) ] == 1, name= "c6_4_" + str(i), )
        model.add_linear_constraint( varsDict["ly_" + str(i) ] + varsDict["wy_" + str(i) ] + varsDict["hy_" + str(i) ] == 1, name= "c6_5_" + str(i), )
        model.add_linear_constraint( varsDict["lz_" + str(i) ] + varsDict["wz_" + str(i) ] + varsDict["hz_" + str(i) ] == 1, name= "c6_6_" + str(i), )



In [37]:
# this prevents to rotate box
for i in range(N):
    if rot_z[i] == 0:
        model.add_linear_constraint ( varsDict["hz_" + str(i) ] == 1, name= "c6_7_" + str(i), )

In [38]:
for j in range(J):
    model.add_linear_constraint( np.sum( [ in_data[i][6]* varsDict["p_" + str(i) + "_" + str(j) ] for i in range(N) ] ) <= bins_info[j, 5] * varsDict["u_" + str(j) ], name= "c7_" + str(j), )